<a href="https://colab.research.google.com/github/towardsai/ai-tutor-rag-system/blob/main/notebooks/Agents_with_OpenAI_Assistants.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [1]:
!pip install -q openai==1.37.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.3 MB/s eta 0:00:00


In [2]:
import os

# Set the "OPENAI_API_KEY" in the Python environment. Will be used by OpenAI client later.
os.environ["OPENAI_API_KEY"] = "[OPENAI_API_KEY]"

# Math Tutor


In [5]:
from openai import OpenAI

client = OpenAI()

assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    model="gpt-4o",
    tools=[{"type": "code_interpreter"}],
)

In [6]:
thread = client.beta.threads.create()

In [7]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `3x + 11 = 14`. Can you help me?",
)

In [8]:
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
)

In [13]:
messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))

In [21]:
print(messages[0].content[0].text.value)

Of course! To solve the equation \(3x + 11 = 14\), follow these steps:

1. **Isolate the variable term**:
   Subtract 11 from both sides of the equation.
   \[
   3x + 11 - 11 = 14 - 11
   \]
   Simplifies to:
   \[
   3x = 3
   \]

2. **Solve for \( x \)**:
   Divide both sides of the equation by 3.
   \[
   x = \frac{3}{3}
   \]
   Simplifies to:
   \[
   x = 1
   \]

So, the solution to the equation is \( x = 1 \).

Let's verify this by substituting \( x = 1 \) back into the original equation to confirm that both sides are equal.

\[
3(1) + 11 = 14
\]
\[
3 + 11 = 14
\]
\[
14 = 14
\]

The left and right sides are equal, so the solution \( x = 1 \) is correct.


# Customer Support


In [3]:
!wget https://personales.unican.es/corcuerp/linux/resources/LinuxCommandLineCheatSheet_1.pdf

--2024-07-23 17:09:57--  https://personales.unican.es/corcuerp/linux/resources/LinuxCommandLineCheatSheet_1.pdf
Resolving personales.unican.es (personales.unican.es)... 193.144.193.111
Connecting to personales.unican.es (personales.unican.es)|193.144.193.111|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 267816 (262K) [application/pdf]
Saving to: ‘LinuxCommandLineCheatSheet_1.pdf’

LinuxCommandLineChe 100%[===================>] 261.54K   314KB/s    in 0.8s    

2024-07-23 17:09:58 (314 KB/s) - ‘LinuxCommandLineCheatSheet_1.pdf’ saved [267816/267816]



In [5]:
from openai import OpenAI

client = OpenAI()

In [6]:
# Create a vector store caled "Financial Statements"
vector_store = client.beta.vector_stores.create(name="Tech Support")

# Ready the files for upload to OpenAI
file_streams = [open("LinuxCommandLineCheatSheet_1.pdf", "rb")]

# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id, files=file_streams
)

# You can print the status and the file counts of the batch to see the result of this operation.
print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)


In [7]:
assistant = client.beta.assistants.create(
    name="Tech Support",
    instructions="You are a tech support chatbot. Use the product manual to respond accurately to customer inquiries.",
    model="gpt-4o",
    tools=[{"type": "file_search"}],
    tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

In [8]:
# Create a thread and attach the file to the message
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "What 'ls' command do?",
        }
    ]
)

In [9]:
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
)

In [10]:
messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))

print(messages[0].content[0].text.value)

The `ls` command in Linux is used to list the contents of a directory. The common usage of `ls` can be extended with options to display detailed information about files and directories. For example:

- `ls -al` lists all files, including hidden ones, in a long listing format that provides detailed information such as permissions, number of links, owner, group, size, and timestamp【4:0†source】【4:1†source】.


In [24]:
messages[0].content[0].text.annotations

[FileCitationAnnotation(end_index=394, file_citation=FileCitation(file_id='file-EMNwQYbq7rGni9Ct4V7B8XTR'), start_index=382, text='【4:0†source】', type='file_citation'),
 FileCitationAnnotation(end_index=406, file_citation=FileCitation(file_id='file-EMNwQYbq7rGni9Ct4V7B8XTR'), start_index=394, text='【4:1†source】', type='file_citation')]